In [12]:
#collect images - code
#setup labelImg
#label images
#update labelmap
#train
#update checkpoint
#detect signs

import os #package to create files and manage filepaths
# !pip install wget #install wget
import wget #to download from urls

# Setup 1
## (no need of executing)

In [4]:
#download tensorflow model garden to use object_dectection api
if not os.path.exists(os.path.join('/Users/asmitabasak/Hand_Gesture/Tensorflow/models', 'research', 'object_detection')):
    !git clone https://github.com/tensorflow/models {'/Users/asmitabasak/Hand_Gesture/Tensorflow/models'}

^C


In [3]:
#install protoc zip
url="https://github.com/protocolbuffers/protobuf/releases/download/v3.15.6/protoc-3.15.6-win64.zip"
wget.download(url)

100% [......................................................] 1468733 / 1468733

'protoc-3.15.6-win64.zip'

In [4]:
#unzip the protoc zip
!mv protoc-3.15.6-win64.zip {'/Users/asmitabasak/Hand_Gesture/Tensorflow/protoc'}
!cd {'/Users/asmitabasak/Hand_Gesture/Tensorflow/protoc'} && unzip protoc-3.15.6-win64.zip

Archive:  protoc-3.15.6-win64.zip
   creating: include/
   creating: include/google/
   creating: include/google/protobuf/
  inflating: include/google/protobuf/wrappers.proto  
  inflating: include/google/protobuf/field_mask.proto  
  inflating: include/google/protobuf/api.proto  
  inflating: include/google/protobuf/struct.proto  
  inflating: include/google/protobuf/descriptor.proto  
  inflating: include/google/protobuf/timestamp.proto  
   creating: include/google/protobuf/compiler/
  inflating: include/google/protobuf/compiler/plugin.proto  
  inflating: include/google/protobuf/empty.proto  
  inflating: include/google/protobuf/any.proto  
  inflating: include/google/protobuf/source_context.proto  
  inflating: include/google/protobuf/type.proto  
  inflating: include/google/protobuf/duration.proto  
   creating: bin/
  inflating: bin/protoc.exe          
  inflating: readme.txt              


# Setup 2

In [5]:
os.environ['PATH'] += os.pathsep + os.path.abspath(os.path.join('D:/Users/morvin/Documents/GitHub/RealTimeObjectDetection/Tensorflow/protoc', 'bin'))

In [6]:
!cd D:/Users/morvin/Documents/GitHub/RealTimeObjectDetection/Tensorflow/models/research && protoc object_detection/protos/*.proto --python_out=. && cp object_detection/packages/tf2/setup.py setup.py && python setup.py build && python setup.py install

object_detection/protos/box_coder.proto: "object_detection.protos.BoxCoder.box_coder_oneof" is already defined in file "object_detection/protos/box_coder(1).proto".
object_detection/protos/box_coder.proto:14:24: "object_detection.protos.BoxCoder.faster_rcnn_box_coder" is already defined in file "object_detection/protos/box_coder(1).proto".
object_detection/protos/box_coder.proto:15:24: "object_detection.protos.BoxCoder.mean_stddev_box_coder" is already defined in file "object_detection/protos/box_coder(1).proto".
object_detection/protos/box_coder.proto:16:20: "object_detection.protos.BoxCoder.square_box_coder" is already defined in file "object_detection/protos/box_coder(1).proto".
object_detection/protos/box_coder.proto:17:22: "object_detection.protos.BoxCoder.keypoint_box_coder" is already defined in file "object_detection/protos/box_coder(1).proto".
object_detection/protos/box_coder.proto:12:9: "object_detection.protos.BoxCoder" is already defined in file "object_detection/protos/bo

In [7]:
#restart kernel
import object_detection

## Setup 3

In [2]:
WORKSPACE_PATH = 'D:/Users/morvin/Documents/GitHub/RealTimeObjectDetection/Tensorflow/workspace'
SCRIPTS_PATH = 'D:/Users/morvin/Documents/GitHub/RealTimeObjectDetection/Tensorflow/scripts'
APIMODEL_PATH = 'D:/Users/morvin/Documents/GitHub/RealTimeObjectDetection/Tensorflow/models'
ANNOTATION_PATH = WORKSPACE_PATH+'/annotations'
IMAGE_PATH = WORKSPACE_PATH+'/images'
MODEL_PATH = WORKSPACE_PATH+'/models'
PRETRAINED_MODEL_PATH = WORKSPACE_PATH+'/pre-trained-models'
CONFIG_PATH = MODEL_PATH+'/my_ssd_mobnet/pipeline.config'
CHECKPOINT_PATH = MODEL_PATH+'/my_ssd_mobnet/'

In [4]:
labels = [{'name':'Hello', 'id':1}, {'name':'Yes', 'id':2}, {'name':'No','id':3}, {'name':'Thank You','id':4}, {'name':'I love you','id':5}]

with open(ANNOTATION_PATH + '/label_map.pbtxt', 'w') as f:
    for label in labels:
        f.write('item { \n')
        f.write('\tname:\'{}\'\n'.format(label['name']))
        f.write('\tid:{}\n'.format(label['id']))
        f.write('}\n')

In [58]:
#create tfrecords
# !python {SCRIPTS_PATH + '/generate_tfrecord.py'} -x {IMAGE_PATH + '/train_gray'} -l {ANNOTATION_PATH + '/label_map.pbtxt'} -o {ANNOTATION_PATH + '/train.record'}
# !python {SCRIPTS_PATH + '/generate_tfrecord.py'} -x {IMAGE_PATH + '/test_gray'} -l {ANNOTATION_PATH + '/label_map.pbtxt'} -o {ANNOTATION_PATH + '/test.record'}

Successfully created the TFRecord file: /Users/asmitabasak/Hand_Gesture/Tensorflow/workspace/annotations/train.record
Successfully created the TFRecord file: /Users/asmitabasak/Hand_Gesture/Tensorflow/workspace/annotations/test.record


In [8]:
#download the model
CUSTOM_MODEL_NAME = 'my_ssd_mobnet' 
# !mkdir {'/Users/asmitabasak/Hand_Gesture/Tensorflow/workspace/models/'+CUSTOM_MODEL_NAME}
# !cp {PRETRAINED_MODEL_PATH+'/ssd_mobilenet_v2_fpnlite_320x320_coco17_tpu-8/pipeline.config'} {MODEL_PATH+'/'+CUSTOM_MODEL_NAME}

## Update config

In [9]:
import tensorflow as tf
from object_detection.utils import config_util
from object_detection.protos import pipeline_pb2
from google.protobuf import text_format

In [7]:
CONFIG_PATH = MODEL_PATH+'/'+CUSTOM_MODEL_NAME+'/pipeline.config'
config = config_util.get_configs_from_pipeline_file(CONFIG_PATH)
config

{'model': ssd {
   num_classes: 90
   image_resizer {
     fixed_shape_resizer {
       height: 320
       width: 320
     }
   }
   feature_extractor {
     type: "ssd_mobilenet_v2_fpn_keras"
     depth_multiplier: 1.0
     min_depth: 16
     conv_hyperparams {
       regularizer {
         l2_regularizer {
           weight: 3.9999998989515007e-05
         }
       }
       initializer {
         random_normal_initializer {
           mean: 0.0
           stddev: 0.009999999776482582
         }
       }
       activation: RELU_6
       batch_norm {
         decay: 0.996999979019165
         scale: true
         epsilon: 0.0010000000474974513
       }
     }
     use_depthwise: true
     override_base_feature_extractor_hyperparams: true
     fpn {
       min_level: 3
       max_level: 7
       additional_layer_depth: 128
     }
   }
   box_coder {
     faster_rcnn_box_coder {
       y_scale: 10.0
       x_scale: 10.0
       height_scale: 5.0
       width_scale: 5.0
     }
   }
   matc

In [8]:
pipeline_config = pipeline_pb2.TrainEvalPipelineConfig()
with tf.io.gfile.GFile(CONFIG_PATH, "r") as f:                                                                                                                                                                                                                     
    proto_str = f.read()                                                                                                                                                                                                                                          
    text_format.Merge(proto_str, pipeline_config) 

In [9]:
pipeline_config.model.ssd.num_classes = 5
pipeline_config.train_config.batch_size = 4
pipeline_config.train_config.fine_tune_checkpoint = PRETRAINED_MODEL_PATH+'/ssd_mobilenet_v2_fpnlite_320x320_coco17_tpu-8/checkpoint/ckpt-0'
pipeline_config.train_config.fine_tune_checkpoint_type = "detection"
pipeline_config.train_input_reader.label_map_path= ANNOTATION_PATH + '/label_map.pbtxt'
pipeline_config.train_input_reader.tf_record_input_reader.input_path[:] = [ANNOTATION_PATH + '/train.record']
pipeline_config.eval_input_reader[0].label_map_path = ANNOTATION_PATH + '/label_map.pbtxt'
pipeline_config.eval_input_reader[0].tf_record_input_reader.input_path[:] = [ANNOTATION_PATH + '/test.record']

In [10]:
config_text = text_format.MessageToString(pipeline_config)                                                                                                                                                                                                        
with tf.io.gfile.GFile(CONFIG_PATH, "wb") as f:                                                                                                                                                                                                                     
    f.write(config_text)

## Training

In [50]:
#for training your own model(on windows)
#!pip install pytz
#!pip install git+https://github.com/philferriere/cocoapi.git#egg=pycocotools^&subdirectory=PythonAPI

In [5]:
command="python {}/research/object_detection/model_main_tf2.py --model_dir={}/{} --pipeline_config_path={}/{}/pipeline.config --num_train_steps=10000".format(APIMODEL_PATH, MODEL_PATH,CUSTOM_MODEL_NAME,MODEL_PATH,CUSTOM_MODEL_NAME)

In [6]:
#!{command}
print(command)

python D:/Users/morvin/Documents/GitHub/RealTimeObjectDetection/Tensorflow/models/research/object_detection/model_main_tf2.py --model_dir=D:/Users/morvin/Documents/GitHub/RealTimeObjectDetection/Tensorflow/workspace/models/my_ssd_mobnet --pipeline_config_path=D:/Users/morvin/Documents/GitHub/RealTimeObjectDetection/Tensorflow/workspace/models/my_ssd_mobnet/pipeline.config --num_train_steps=10000


## Loading checkpoint

In [10]:
from object_detection.utils import label_map_util
from object_detection.utils import visualization_utils as viz_utils
from object_detection.builders import model_builder

In [13]:
# Load pipeline config and build a detection model
configs = config_util.get_configs_from_pipeline_file(CONFIG_PATH)
detection_model = model_builder.build(model_config=configs['model'], is_training=False)

# Restore checkpoint
ckpt = tf.compat.v2.train.Checkpoint(model=detection_model)
ckpt.restore(os.path.join(CHECKPOINT_PATH, 'ckpt-10')).expect_partial()

@tf.function
def detect_fn(image):
    image, shapes = detection_model.preprocess(image)
    prediction_dict = detection_model.predict(image, shapes)
    detections = detection_model.postprocess(prediction_dict, shapes)
    return detections
#detect_fn = tf.function(detect_fn(image))

## Live Detection

In [14]:
import cv2 
import numpy as np

In [15]:
category_index = label_map_util.create_category_index_from_labelmap(ANNOTATION_PATH+'/label_map.pbtxt')


In [16]:
# Setup capture
cap = cv2.VideoCapture(0)
width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
while True: 
    ret, frame = cap.read()
    image_np = np.array(frame)
    
    input_tensor = tf.convert_to_tensor(np.expand_dims(image_np, 0), dtype=tf.float32)
    detections = detect_fn(input_tensor)
    #print(detections)

    num_detections = int(detections.pop('num_detections'))
    
    detections = {key: value[0, :num_detections].numpy()
                  for key, value in detections.items()}
    detections['num_detections'] = num_detections

    # detection_classes should be ints.
    #print(detections['detection_classes'])
    detections['detection_classes'] = detections['detection_classes'].astype(np.int64)

    label_id_offset = 1
    image_np_with_detections = image_np.copy()
    
    viz_utils.visualize_boxes_and_labels_on_image_array(
                image_np_with_detections,
                detections['detection_boxes'],
                detections['detection_classes']+label_id_offset,
                detections['detection_scores'],
                category_index,
                use_normalized_coordinates=True,
                max_boxes_to_draw=1,
                min_score_thresh=.6,
                agnostic_mode=False)
    viz_utils.visualize_boxes_and_labels_on_image_array(
                image_np_with_detections,
                detections['detection_boxes'],
                detections['detection_classes']+label_id_offset,
                detections['detection_scores'],
                category_index,
                use_normalized_coordinates=False,
                max_boxes_to_draw=1,
                min_score_thresh=.6,
                agnostic_mode=False)
#     print(detections["detection_scores"])

#     print(detections['detection_boxes'])
    cv2.imshow('object detection',  cv2.resize(image_np_with_detections, (800, 600)))
    
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break
cap.release()
cv2.destroyAllWindows()
for i in range(5):
    cv2.waitKey(1)

In [15]:
cap.release()

In [20]:
# img=cv2.imread('/Users/asmitabasak/Downloads/laterite.webp')
# cv2.imshow('frame',img)